In [1]:
# Cell 1: Imports and Setup
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
import ipywidgets as widgets
from ipywidgets import interactive, VBox, HBox, HTML, Layout

# Set a default style for plots
plt.style.use('seaborn-v0_8-whitegrid')

# Cell 2: Configuration (User Input)
# --- Set the observed number of counts here ---
observed_counts_widget = widgets.IntText(
    value=9,  # Default observed counts
    description='Observed Counts (x):',
    disabled=False,
    style={'description_width': 'initial'}
)

# --- Set the desired confidence level here ---
confidence_level_widget = widgets.Dropdown(
    options=[('90%', 0.90), ('95%', 0.95), ('99%', 0.99)],
    value=0.90, # Default CL
    description='Confidence Level (1-α):',
    disabled=False,
    style={'description_width': 'initial'}
)

# Display the widgets for configuration
config_box = VBox([observed_counts_widget, confidence_level_widget])
display(config_box)

# Cell 3: Define the plotting function
def plot_poisson_tails(x, alpha, lambda_lower_candidate, lambda_upper_candidate):
    """
    Calculates tail probabilities and plots Poisson distributions
    for candidate lower and upper lambda values.
    """
    target_prob = alpha / 2.0

    # --- Calculations ---
    # Lower bound check: P(X >= x | lambda_lower_candidate)
    # sf(k, mu) is P(X > k), so P(X >= x) = P(X > x-1) = sf(x-1, lambda_lower_candidate)
    if lambda_lower_candidate <= 0:
         prob_ge_x = 1.0 # P(X>=x | lambda=0) is 1 if x=0, 0 otherwise. Handle x=0 separately
         if x == 0:
             prob_ge_x = 1.0
         else:
             # Technically undefined or limit is 0, but sf handles it.
             # Let's avoid lambda=0 for sf calculation practicality if x > 0
             prob_ge_x = poisson.sf(x - 1, 1e-9) if x > 0 else 1.0
    else:
         prob_ge_x = poisson.sf(x - 1, lambda_lower_candidate)

    # Upper bound check: P(X <= x | lambda_upper_candidate)
    # cdf(k, mu) is P(X <= k)
    if lambda_upper_candidate <= 0:
         # Avoid issues with lambda=0 if x isn't 0
         prob_le_x = poisson.cdf(x, 1e-9) if x > 0 else 1.0
    else:
        prob_le_x = poisson.cdf(x, lambda_upper_candidate)

    # --- Plotting ---
    fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

    # Plot for Lower Bound Check
    ax_lower = axes[0]
    # Determine plot range for k (number of counts)
    k_max_lower = int(max(x + 5, lambda_lower_candidate + 4 * np.sqrt(max(1, lambda_lower_candidate))))
    k_values_lower = np.arange(0, k_max_lower + 1)
    pmf_lower = poisson.pmf(k_values_lower, lambda_lower_candidate)

    bars_lower = ax_lower.bar(k_values_lower, pmf_lower, label=f'Poisson PMF (λ={lambda_lower_candidate:.2f})', alpha=0.6, color='grey')
    # Highlight tail P(X >= x)
    tail_indices_lower = k_values_lower >= x
    ax_lower.bar(k_values_lower[tail_indices_lower], pmf_lower[tail_indices_lower], color='red', alpha=0.8, label=f'P(X ≥ {x})')
    ax_lower.set_title(f'Lower Bound Check (λ_lower = {lambda_lower_candidate:.3f})\nTarget P(X ≥ {x}) = {target_prob:.3f}')
    ax_lower.text(0.65, 0.65, f'Actual P(X ≥ {x}) = {prob_ge_x:.4f}',
                  ha='right', va='top', transform=ax_lower.transAxes,
                  bbox=dict(boxstyle='round,pad=0.3', fc='wheat', alpha=0.8))
    ax_lower.set_xlabel('Number of Counts (k)')
    ax_lower.set_ylabel('Probability Mass P(k | λ)')
    ax_lower.legend(loc='upper right')
    ax_lower.grid(True, axis='y')


    # Plot for Upper Bound Check
    ax_upper = axes[1]
     # Determine plot range for k (number of counts)
    k_max_upper = int(max(x + 5, lambda_upper_candidate + 4 * np.sqrt(max(1, lambda_upper_candidate))))
    k_values_upper = np.arange(0, k_max_upper + 1)
    pmf_upper = poisson.pmf(k_values_upper, lambda_upper_candidate)

    bars_upper = ax_upper.bar(k_values_upper, pmf_upper, label=f'Poisson PMF (λ={lambda_upper_candidate:.2f})', alpha=0.6, color='grey')
    # Highlight tail P(X <= x)
    tail_indices_upper = k_values_upper <= x
    ax_upper.bar(k_values_upper[tail_indices_upper], pmf_upper[tail_indices_upper], color='red', alpha=0.8, label=f'P(X ≤ {x})')
    ax_upper.set_title(f'Upper Bound Check (λ_upper = {lambda_upper_candidate:.3f})\nTarget P(X ≤ {x}) = {target_prob:.3f}')
    ax_upper.text(0.65, 0.65, f'Actual P(X ≤ {x}) = {prob_le_x:.4f}',
                  ha='right', va='top', transform=ax_upper.transAxes,
                   bbox=dict(boxstyle='round,pad=0.3', fc='wheat', alpha=0.8))
    ax_upper.set_xlabel('Number of Counts (k)')
    # ax_upper.set_ylabel('Probability Mass P(k | λ)') # Shared Y
    ax_upper.legend(loc='upper right')
    ax_upper.grid(True, axis='y')

    # Add vertical line at x
    for ax in axes:
        ymin, ymax = ax.get_ylim()
        ax.vlines(x, ymin, ymax, color='black', linestyle='--', lw=1.5, label=f'Observed x = {x}')
        ax.legend(loc='upper right') # Re-call legend to include vline label if needed, adjust loc
        ax.set_ylim(ymin, ymax) # Keep ylim consistent

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
    plt.show()

# Cell 4: Create Interactive Sliders and Link to Plot
# Get initial values from widgets
initial_x = observed_counts_widget.value
initial_cl = confidence_level_widget.value
initial_alpha = 1.0 - initial_cl

# --- Determine sensible slider ranges based on x ---
# These are heuristic ranges, might need adjustment for extreme x values
# Lower bound is typically <= x, Upper bound is typically >= x
# Add small epsilon to avoid lambda=0 issues in some calculations if needed
min_lambda = 0.01

# Heuristic range finding (can be improved)
approx_stderr = np.sqrt(max(1, initial_x)) # Use max(1,x) for sqrt stability if x=0
slider_lower_min = max(min_lambda, initial_x - 5 * approx_stderr)
slider_lower_max = max(min_lambda + 0.1, initial_x + 2 * approx_stderr) # Lower lambda shouldn't exceed x much
slider_lower_start = max(min_lambda, initial_x - 1 * approx_stderr) # Start guess

slider_upper_min = max(min_lambda, initial_x - 2 * approx_stderr) # Upper lambda can be < x
slider_upper_max = max(min_lambda + 0.1, initial_x + 5 * approx_stderr)
slider_upper_start = max(min_lambda, initial_x + 1 * approx_stderr) # Start guess

# Ensure max > min
if slider_lower_max <= slider_lower_min: slider_lower_max = slider_lower_min + 1
if slider_upper_max <= slider_upper_min: slider_upper_max = slider_upper_min + 1
# Ensure start is within bounds
slider_lower_start = np.clip(slider_lower_start, slider_lower_min, slider_lower_max)
slider_upper_start = np.clip(slider_upper_start, slider_upper_min, slider_upper_max)


# Create sliders
slider_lambda_lower = widgets.FloatSlider(
    value=slider_lower_start,
    min=slider_lower_min,
    max=slider_lower_max,
    step=0.01,
    description='λ_lower guess:',
    continuous_update=False, # Only update plot when slider released
    readout=True,
    readout_format='.3f',
    layout=Layout(width='80%')
)

slider_lambda_upper = widgets.FloatSlider(
    value=slider_upper_start,
    min=slider_upper_min,
    max=slider_upper_max,
    step=0.01,
    description='λ_upper guess:',
    continuous_update=False,
    readout=True,
    readout_format='.3f',
    layout=Layout(width='80%')
)

# Create the interactive output
interactive_plot = interactive(
    plot_poisson_tails,
    # Pass fixed values from widgets using .value
    x=widgets.fixed(observed_counts_widget.value),
    alpha=widgets.fixed(1.0 - confidence_level_widget.value),
    # Link sliders
    lambda_lower_candidate=slider_lambda_lower,
    lambda_upper_candidate=slider_lambda_upper
)

# Cell 5: Instructions and Display Interactive Elements
instruction_html = HTML(
    value=f"""
    <h3>Interactive Poisson Confidence Interval Finder</h3>
    <p>Based on observing <b>x = {observed_counts_widget.value}</b> counts, we want to find the <b>{confidence_level_widget.value*100:.0f}%</b> confidence interval [λ_lower, λ_upper] for the true mean λ.</p>
    <p>This interval is defined such that:</p>
    <ul>
        <li><b>Lower Bound (λ_lower):</b> P(X ≥ {observed_counts_widget.value} | λ=λ_lower) = α/2 = {(1.0-confidence_level_widget.value)/2.0:.3f}</li>
        <li><b>Upper Bound (λ_upper):</b> P(X ≤ {observed_counts_widget.value} | λ=λ_upper) = α/2 = {(1.0-confidence_level_widget.value)/2.0:.3f}</li>
    </ul>
    <p><b>Your Task:</b> Adjust the sliders below. </p>
    <ul>
    <li>Use the <b>top slider (λ_lower guess)</b> until the 'Actual P(X ≥ {observed_counts_widget.value})' value in the <b>left plot</b> is as close as possible to <b>{(1.0-confidence_level_widget.value)/2.0:.3f}</b>.</li>
    <li>Use the <b>bottom slider (λ_upper guess)</b> until the 'Actual P(X ≤ {observed_counts_widget.value})' value in the <b>right plot</b> is as close as possible to <b>{(1.0-confidence_level_widget.value)/2.0:.3f}</b>.</li>
    </ul>
    <p>The slider values that achieve these target probabilities are the approximate lower and upper bounds of your confidence interval.</p>
    <hr>
    """
)

# Display instructions and interactive plot
display(instruction_html)
display(VBox([slider_lambda_lower, slider_lambda_upper])) # Display sliders separately if needed
display(interactive_plot.children[-1]) # Display the output plot area


# Cell 6: (Optional) Calculate and Display Exact Values for Comparison
from scipy.stats import chi2

def calculate_exact_poisson_ci(x, alpha):
    """Calculates the exact Poisson CI using the Chi-squared method."""
    if x == 0:
        lambda_lower = 0.0
    else:
        lambda_lower = 0.5 * chi2.ppf(alpha / 2, 2 * x)

    lambda_upper = 0.5 * chi2.ppf(1 - alpha / 2, 2 * (x + 1))
    return lambda_lower, lambda_upper

# Get current values
current_x = observed_counts_widget.value
current_alpha = 1.0 - confidence_level_widget.value
current_cl = confidence_level_widget.value * 100

exact_lower, exact_upper = calculate_exact_poisson_ci(current_x, current_alpha)

print("\n--- Exact Calculation (for comparison) ---")
print(f"Observed counts (x): {current_x}")
print(f"Confidence Level: {current_cl:.0f}% (α = {current_alpha:.2f})")
print(f"Exact {current_cl:.0f}% Confidence Interval for λ:")
print(f"  Lower bound = {exact_lower:.4f}")
print(f"  Upper bound = {exact_upper:.4f}")
print(f"Interval: [{exact_lower:.4f}, {exact_upper:.4f}]")


HTML(value="\n    <h3>Interactive Poisson Confidence Interval Finder</h3>\n    <p>Based on observing <b>x = 9<…

Output()


--- Exact Calculation (for comparison) ---
Observed counts (x): 9
Confidence Level: 90% (α = 0.10)
Exact 90% Confidence Interval for λ:
  Lower bound = 4.6952
  Upper bound = 15.7052
Interval: [4.6952, 15.7052]
